In [11]:
import numpy as np
import pandas as pd
from importlib import reload
from copy import deepcopy as copy

from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array, as_float_array
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier

from bayes_opt import BayesianOptimization

from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector, ExhaustiveFeatureSelector

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('../')

In [4]:
n_holdout_interations = 100

In [5]:
df = pd.read_csv('./classification/datasets/cancer/breast_cancer.csv')
y = df.diagnosis.replace('M', 0).replace('B', 1).values
X = np.asarray(df.drop(['diagnosis', 'id', 'Unnamed: 32'], axis=1).as_matrix())

In [6]:
X = StandardScaler().fit_transform(X)

In [7]:
models = {
    'lr': LogisticRegression(random_state=42, multi_class='ovr', solver='liblinear', C=10000, tol=1e-2),
    'svc': LinearSVC(multi_class='ovr', C=10000, tol=1e-2, max_iter=250),
}

# SequentialFeatureSelection

In [13]:
model_feats = dict()

for k, v in models.items():
    print('model type: {}'.format(k))
    sfs = SequentialFeatureSelector(v, k_features=10, scoring='accuracy')
    sfs.fit(X, y)
    
    model_feats[k] = [int(i) for i in sfs.subsets_[5]['feature_names']]

model type: lr
model type: svc


In [20]:
model_feats

{'lr': [4, 7, 21, 22, 24], 'svc': [1, 6, 23, 24, 28]}

In [14]:
accur_score = list()

for i in range(n_holdout_interations):
    record = dict()
    for name, model in models.items():
        X_train, X_val, y_train, y_val = train_test_split(X[:, model_feats[name]], y, test_size=0.5)#, random_state=1)
        
        model.fit(X_train, y_train.squeeze())
        record[name] = accuracy_score(model.predict(X_val), y_val.squeeze())
    accur_score.append(record)

print(np.mean([i['svc'] for i in accur_score]))
print(np.mean([i['lr'] for i in accur_score]))

0.9355438596491229
0.9722105263157895


# ExhaustiveFeatureSelection

In [24]:
efs = ExhaustiveFeatureSelector(estimator=models['lr'], max_features=5, scoring='accuracy', n_jobs=-1, print_progress=False)
efs.fit(X, y)

ExhaustiveFeatureSelector(clone_estimator=True, cv=5,
             estimator=LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.01, verbose=0, warm_start=False),
             max_features=5, min_features=1, n_jobs=-1,
             pre_dispatch='2*n_jobs', print_progress=False,
             scoring='accuracy')

In [25]:
efs.best_feature_names_

('17', '21', '23', '24', '28')

In [29]:
accur_score = list()

for i in range(n_holdout_interations):
    record = dict()
    X_train, X_val, y_train, y_val = train_test_split(X[:, [int(i) for i in efs.best_feature_names_]], y, test_size=0.5)#, random_state=1)
        
    models['lr'].fit(X_train, y_train.squeeze())
    record['lr'] = accuracy_score(models['lr'].predict(X_val), y_val.squeeze())
    accur_score.append(record)

print(np.mean([i['lr'] for i in accur_score]))

0.9732280701754387


# RecursiveFeatureElimination

In [21]:
model_feats = dict()

for k, v in models.items():
    model_feats[k] = [int(i) for i in sfs.subsets_[5]['feature_names']]
    rfe = RFE(estimator=v, n_features_to_select=5)
    rfe.fit(X, y)
    model_feats[k] = np.array(list(range(X.shape[1])))[rfe.support_]

In [22]:
model_feats

{'lr': array([10, 20, 21, 23, 27]), 'svc': array([ 0,  6, 13, 19, 23])}

In [23]:
accur_score = list()

for i in range(n_holdout_interations):
    record = dict()
    for name, model in models.items():
        X_train, X_val, y_train, y_val = train_test_split(X[:, model_feats[name]], y, test_size=0.5)#, random_state=1)
        
        model.fit(X_train, y_train.squeeze())
        record[name] = accuracy_score(model.predict(X_val), y_val.squeeze())
    accur_score.append(record)

print(np.mean([i['svc'] for i in accur_score]))
print(np.mean([i['lr'] for i in accur_score]))

0.9130877192982457
0.9646666666666667


# Logistic Regression coefficents

In [17]:
model_coefs = []

c = copy(models['lr'])
c.fit(X, y)
    
model_coefs = np.abs(c.coef_).sum(0)

In [19]:
sorted(zip(model_coefs.tolist(), range(model_coefs.shape[0])), key = lambda x: -x[0])[:5]

[(1.8956650592717343, 10),
 (1.7403622083957977, 21),
 (1.3205567864813075, 20),
 (1.3197392480935726, 28),
 (1.2445662612696033, 13)]